In [2]:
from scripts.experiments.symmetries import pauli_reduce, clifford_symmetry
from sympleq.models.Ising import ising_chain_hamiltonian, ising_2d_hamiltonian

In [2]:



N = 5
J = 1
h = 0.5
H = ising_chain_hamiltonian(N, J, h, periodic=True)

h_red, conditioned_hamiltonians, C, all_phases = pauli_reduce(H)
print(h_red)
print(conditioned_hamiltonians[0])


(1+0j)  |x0z1 x0z0 x0z0 x0z0 x0z0 | 0 
(1+0j)  |x0z0 x0z1 x0z0 x0z0 x0z0 | 0 
(1+0j)  |x0z0 x0z0 x0z1 x0z0 x0z0 | 0 
(1+0j)  |x0z0 x0z0 x0z0 x0z1 x0z0 | 0 
(1+0j)  |x0z1 x0z1 x0z1 x0z1 x0z0 | 0 
(0.5+0j)|x1z0 x0z0 x0z0 x0z0 x0z0 | 0 
(0.5+0j)|x1z0 x1z0 x0z0 x0z0 x0z0 | 0 
(0.5+0j)|x0z0 x1z0 x1z0 x0z0 x0z0 | 0 
(0.5+0j)|x0z0 x0z0 x1z0 x1z0 x0z0 | 0 
(0.5+0j)|x0z0 x0z0 x0z0 x1z0 x0z1 | 0 

(0.5+0j)|x1z0 x1z0 x0z0 x0z0 | 0 
(0.5+0j)|x1z0 x0z0 x0z0 x0z0 | 0 
(0.5+0j)|x0z0 x1z0 x1z0 x0z0 | 0 
(0.5+0j)|x0z0 x0z0 x1z0 x1z0 | 0 
(0.5+0j)|x0z0 x0z0 x0z0 x1z0 | 0 
(1+0j)  |x0z1 x0z1 x0z1 x0z1 | 0 
(1+0j)  |x0z1 x0z0 x0z0 x0z0 | 0 
(1+0j)  |x0z0 x0z1 x0z0 x0z0 | 0 
(1+0j)  |x0z0 x0z0 x0z1 x0z0 | 0 
(1+0j)  |x0z0 x0z0 x0z0 x0z1 | 0 



In [3]:
F = clifford_symmetry(conditioned_hamiltonians[0])
print(F.symplectic)


[0 0 0 0 0 0 0 0 0 0]
[[0 1 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0]
 [1 1 1 1 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 0 1 1 0]
 [0 0 0 0 1 0 1 0]
 [0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 1 1]]


In [3]:
n_x = 8
n_y = 7
J_zz = 1
h_x = 0.5
H_2d = ising_2d_hamiltonian(n_x, n_y, J_zz, h_x, periodic=False)

h_red, conditioned_hamiltonians, C, all_phases = pauli_reduce(H_2d)
print(conditioned_hamiltonians[0].n_qudits())
print(conditioned_hamiltonians[0].n_paulis())

55
153


In [ ]:
F = clifford_symmetry(conditioned_hamiltonians[0])


In [8]:
print(F.symplectic)

[[0 1 1 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 1]
 [0 0 0 ... 1 1 0]]
